In [ ]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np 
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from collections import defaultdict
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv("temp_data3.csv")
df=df.iloc[:4000,:]

In [ ]:
list1=[]
list1=df.Sentiment.values
for i in range(len(list1)):
    if(list1[i]=="mixed"):
        df.drop([i],axis=0,inplace=True)
    elif(list1[i]=="neutral"):
        df.drop([i],axis=0,inplace=True)

df["Sentiment"].value_counts()

In [ ]:
text=df["cleaned_text"]
sentiment=df["Sentiment"]
dataf=pd.concat([text,sentiment],axis=1)

In [ ]:
data=dataf["cleaned_text"].apply(lambda x: " ".join(x.lower() for x in x.split() if x.isalpha()))
data=data.str.replace("[^\w\s]","")#harf dışı simgeler,noktalama
data= data.str.replace(" \d+", " ")#sayılar 
from nltk.corpus import stopwords
nltk.download("stopwords")
duraklama_kelimeleri = stopwords.words()
data=pd.DataFrame(data,columns=["cleaned_text"])
data=data["cleaned_text"].apply(lambda x: " ".join(x for x in x.split() if x not in duraklama_kelimeleri))
data=pd.concat([data,sentiment],axis=1)

In [ ]:
cleaned_text = data['cleaned_text'].values
labels = data['Sentiment'].values
cleaned_text=list(cleaned_text)

In [ ]:
def IDF(corpus, unique_words):
    idf_dict={}
    N=len(corpus)
    for i in unique_words:
        count=0
        for sen in corpus:
            if i in sen.split():
                count=count+1#kelimenin kaç dökümanda olduğu
            idf_dict[i]=(math.log((1+N)/(count+1)))+1
    return idf_dict 

In [ ]:
def fit(whole_data):
    unique_words = set()
    if isinstance(whole_data, (list,)):
        for x in whole_data:
            for y in x.split():#cümle kelimeleri
                if len(y)<2:
                    continue
                unique_words.add(y)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}#kelime indexleme
    Idf_values_of_all_unique_words=IDF(whole_data,unique_words)
    return vocab, Idf_values_of_all_unique_words
Vocabulary, idf_of_vocabulary=fit(cleaned_text) 

In [ ]:
def transform(dataset,vocabulary,idf_values):
    tfidf=np.zeros(len(vocabulary))
    number_of_words_in_sentence=Counter(dataset.split())#yorumdaki kelimelerin sayısı
    for word in dataset.split():
        if word in  list(vocabulary.keys()):#(kelimenin yorumdaki sayısı/yorumun uzunluğu)*kelimenin idf değeri
            tf_idf_value=(number_of_words_in_sentence[word]/len(dataset.split()))*(idf_values[word])
            tfidf[vocabulary[word]]=tf_idf_value
    return tfidf

In [ ]:
tablo = np.zeros((len(cleaned_text),len(Vocabulary)))
tablo =pd.DataFrame(tablo,columns=Vocabulary.keys())
tfidf=np.zeros(len(Vocabulary))
for i in range(len(cleaned_text)):
    tfidf=transform(cleaned_text[i],Vocabulary,idf_of_vocabulary)
    tablo.loc[i] = tfidf

In [ ]:
tablo=normalize(tablo, norm='l2', axis=1, copy=True, return_norm=False)
tablo =pd.DataFrame(tablo,columns=Vocabulary.keys())

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
train_sentences, test_sentences, train_labels, test_labels = train_test_split(tablo, encoded_labels, stratify = labels)

In [ ]:
def sigmoid(input):    
    output = 1 / (1 + np.exp(-input))
    return output

In [ ]:
def loss(y, y_hat):
    loss = -np.mean(y*(np.log(y_hat)) + (1-y)*np.log(1-y_hat))
    return loss

In [ ]:
def gradients(X, y, y_hat):
    
    m = X.shape[0]
    
    dw = (1/m)*np.dot(X.T, (y_hat - y))
    
    db = (1/m)*np.sum((y_hat - y)) 
    
    return dw, db

In [ ]:
def train(X, y, bs, epochs, lr):
    

    m, n = X.shape
    
    w = np.zeros((n,1))
    b = 0
    
    y = y.reshape(m,1)

    losses=[]
    losses2=[]
    accuracy_tr=[]
    accuracy_te=[]
    iterations2=[]
    weight_dict = {}
    bias_dict={}
    
    for epoch in range(epochs):
        weight_dict[epoch] = defaultdict(lambda: 0)
        bias_dict[epoch] = defaultdict(lambda: 0)
        for i in range((m-1)//bs + 1):
           
            start_i = i*bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]
            
            y_hat = sigmoid(np.dot(xb, w) + b)
            
            dw, db = gradients(xb, yb, y_hat)
            
            w -= lr*dw
            b -= lr*db
            weight_dict[epoch][i]=w
            bias_dict[epoch][i]=b

        l = loss(y, sigmoid(np.dot(X, w) + b))
        losses.append(l)
        pred=predict(X,w,b)
        accuracy_tr.append(accuracy_score(y,pred))
        
        l2 = loss(test_labels, sigmoid(np.dot(test_sentences, w) + b))
        losses2.append(l2)
        pred2=predict(test_sentences,w,b)
        accuracy_te.append(accuracy_score(test_labels,pred2))
        iterations2.append(epoch)
        
    return w, b, dw, db, losses,losses2,accuracy_tr,accuracy_te,iterations2,weight_dict,bias_dict

In [ ]:
def predict(X,w,b):
    

    preds = sigmoid(np.dot(X, w) + b)
    pred_class = []
    pred_class = [1 if i > 0.5 else 0 for i in preds]
    
    return np.array(pred_class)

In [ ]:
w, b, dw, db,losses,losses2,accuracy_tr,accuracy_te,iterations2,weight_dict,bias_dict = train(train_sentences, train_labels, bs=100, epochs=300, lr=0.02)

In [ ]:
b , db

In [ ]:
w.shape,dw.shape

In [ ]:
bias_dict

In [ ]:
weight_dict

In [ ]:
plt.scatter(iterations2,accuracy_tr)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'accuracy_tr' )  
plt.title("0.02-300-TfIdf-Acc_Tr")
#plt.savefig("0.02-500-TfIdf-Acc_Tr.png")
plt.show()

In [ ]:
plt.scatter(iterations2,accuracy_te)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'accuracy_te' )  
plt.title("0.02-300-TfIdf-Acc_Te")
#plt.savefig("0.02-500-TfIdf-Acc_Tr.png")
plt.show()

In [ ]:
plt.scatter(iterations2,losses)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'losses' )  
plt.title("0.02-300-TfIdf-Loss-Tr")
#plt.savefig("0.02-500-TfIdf-Acc_Tr.png")
plt.show()

In [ ]:
plt.scatter(iterations2,losses2)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'losses2' )  
plt.title("0.02-300-TfIdf-Loss-Te")
#plt.savefig("0.02-500-TfIdf-Acc_Tr.png")
plt.show()

In [ ]:
pred=predict(test_sentences,w,b)
accuracy_score(test_labels,pred)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

class_names=["0","1"]

print(classification_report(test_labels, pred, target_names=class_names))

print(confusion_matrix(test_labels, pred))